In [ ]:
import re
import random
import time
from datetime import datetime


class Account:
    INTEREST_RATE = 0.001  # 0.1% 씩 이자 넣기 (주의: 0.1% = 0.001, 10%는 0.1)

    def __init__(self, nickname, password, balance=0, account_number=None):
        # Class에서 받을 정보 넣기, 외부 정보인 경우 self 뒤에 변수 넣어야함
        self.bank_name = "SC 은행"  # 은행 명
        self.nickname = nickname  # 통장별명
        self.password = password  # 패스워드
        self.account_number = account_number or self.generate_account_number()  # 랜덤으로 발생하는 계좌 번호
        self.balance = balance  # 계좌 잔액 인스턴스
        self.history = []  # list of tuples: (type, amount, balance_after, datetime) # 계좌 내역

    def generate_account_number(self):
        # 계좌 생성: 3-2-6 형태 랜덤
        part1 = str(random.randint(100, 999))  # 처음 3자리
        part2 = str(random.randint(10, 99))  # 가운데 2자리
        part3 = str(random.randint(100000, 999999))  # 마지막 6자리
        return f"{part1}-{part2}-{part3}"  # 모두 합쳐서 리턴

    def show_receipt(self, type_, amount, other_account=None):
        # 거래 후 결과값(영수증 스타일 출력)
        now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        print("\n=============================")
        print("📄 거래 결과")
        print("-----------------------------")
        print(f"거래 유형 : {type_}")  # 입금or 출금
        print(f"거래 금액 : {amount:,}원")  # 거래한 금액 자체
        print(f"거래 계좌 : {self.nickname} ({self.account_number})")  # 내 계좌&계좌번호
        if other_account:
            print(f"대상 계좌 : {other_account.nickname} ({other_account.account_number})")  # 상대 계좌
        print(f"잔액 : {self.balance:,}원")  # 잔고
        print(f"거래 일시 : {now}")  # 시간
        print("=============================\n")

    def deposit(self, amount, from_account=None):
        # 입금할 때 처리
        self.balance += amount  # 현재잔고 + 입금 금액
        self.history.append(("입금", amount, self.balance, datetime.now()))  # 거래내역 기록
        self.show_receipt("입금", amount, from_account)  # 거래내역 출력

    def withdraw(self, amount, to_account=None):
        # 출금(송금) 처리
        if amount > self.balance:  # 잔액 부족 방지
            print("❌ 잔액이 부족합니다.")
            return False
        self.balance -= amount  # 출금 처리
        self.history.append(("출금", amount, self.balance, datetime.now()))  # 거래내역 기록
        self.show_receipt("출금", amount, to_account)  # 영수증 출력 (to_account는 상대)
        return True

    def add_interest(self):
        # 이자 부분: 잔고 기준 이자 계산 및 반영
        interest = int(self.balance * Account.INTEREST_RATE)  # 잔고 * 이자율
        if interest <= 0:  # 이자가 0원 이하면 무시
            return
        self.balance += interest  # 잔고에 이자 반영
        self.history.append(("이자 지급", interest, self.balance, datetime.now()))  # 거래내역 기록
        print(f"🎉 축하합니다. {self.nickname}님의 소중한 계좌에 이자를 쏙 넣어드렸어요.")
        self.show_receipt("이자 지급", interest)  # 이자 영수증 출력

    def show_history(self):
        # 거래 내역 보기: 시간순으로 출력
        print(f"\n📄 {self.nickname} 거래 내역")
        if not self.history:  # 거래 내역이 없으면 메시지
            print("거래 내역이 없습니다.")
        for t, a, b, d in self.history:  # t=유형, a=금액, b=잔액, d=시간
            print(f"{d.strftime('%Y-%m-%d %H:%M:%S')} | {t}: {a:,}원 | 잔고: {b:,}원")
        print()


class BankSystem:
    def __init__(self):
        self.accounts = []  # 빈 리스트: 계좌 저장
        self.deposit_count = 0  # 입금 카운트(이자 지급 조건용)
        # 기본 계좌(예시) — 필요하면 주석 처리하거나 제거 가능
        base = Account("기본계좌", "0000", balance=100000, account_number="123-45-678900")
        self.accounts.append(base)

    def create_account(self):
        # 계좌 개설 UI & 로직
        nickname = input("통장 별명을 입력하세요: ").strip()  # 공백 제거
        if not nickname:
            print("통장 별명은 비워둘 수 없습니다.")
            return
        pw = input("비밀번호를 입력하세요: ").strip()
        confirm_pw = input("비밀번호를 다시 입력하세요: ").strip()

        if pw != confirm_pw:
            print("❌ 비밀번호가 일치하지 않습니다. 다시 시도해주세요.\n")
            return

        new_acc = Account(nickname, pw)  # Account 인스턴스 생성
        self.accounts.append(new_acc)  # 리스트에 추가

        print("\n✅ 계좌가 개설되었습니다!")
        print(f"은행: {new_acc.bank_name}")
        print(f"통장별명: {new_acc.nickname}")
        print(f"계좌번호: {new_acc.account_number}")
        print(f"잔고: {new_acc.balance:,}원\n")

    def verify_password(self, account):
        # 비밀번호 확인 함수
        pw = input(f"🔐 [{account.nickname}] 비밀번호를 입력하세요: ").strip()
        if pw == account.password:
            return True
        print("❌ 비밀번호가 일치하지 않습니다.")
        return False

    def show_main_menu(self):
        # 메인 메뉴 루프
        while True:
            print("\n===== [메인 화면] =====")

            # 계좌가 1개 이상 있는지 확인 (없으면 생성 유도)
            if len(self.accounts) == 0:
                print("아직 계좌가 없습니다. 계좌를 먼저 개설하세요.\n")
                self.create_account()
                continue

            print("\n[보유 계좌 목록]")
            for i, acc in enumerate(self.accounts, start=1):
                print(f"{i}. {acc.nickname} ({acc.account_number}) - 잔고: {acc.balance:,}원")

            print("\n메뉴: 1. 계좌 내역  2. 입금(기존→선택)  3. 출금(선택→기본)  4. 신규 계좌 개설  5. 종료")
            choice = input("선택: ").strip()

            if choice == "1":
                self.show_account_history()
            elif choice == "2":
                self.deposit_process()
            elif choice == "3":
                self.withdraw_process()
            elif choice == "4":
                self.create_account()
            elif choice == "5":
                print("👋 프로그램을 종료합니다.")
                break
            else:
                print("잘못된 입력입니다. 다시 선택하세요.")

    def select_account(self, action="선택"):
        # 계좌 선택 UI: 번호 입력으로 Account 객체 반환
        for i, acc in enumerate(self.accounts, start=1):
            print(f"{i}. {acc.nickname} ({acc.account_number}) - 잔고: {acc.balance:,}원")
        try:
            idx = int(input(f"\n어떤 계좌에 {action}하시겠어요? 번호 입력 → ").strip()) - 1
            if idx < 0 or idx >= len(self.accounts):
                print("❌ 잘못된 번호입니다.")
                return None
            return self.accounts[idx]
        except ValueError:
            print("❌ 숫자를 입력해야 합니다.")
            return None

    def clean_amount_input(self, text):
        # 금액 입력 정제: "1,000원" -> 1000 형태로 반환
        text = text.strip()
        if not text:
            return None
        parts = re.findall(r"\d+", text)  # 숫자부분만 추출
        if not parts:
            return None
        try:
            return int("".join(parts))
        except ValueError:
            return None

    def deposit_process(self):
        # 입금 흐름: 기본계좌 -> 선택계좌
        base_acc = self.accounts[0]
        print("\n[입금] — 기본 계좌에서 선택한 계좌로 송금합니다.")
        target_acc = self.select_account("입금")
        if not target_acc:
            return
        # 받는 계좌의 비밀번호 확인 (설계상 변경 가능)
        if not self.verify_password(target_acc):
            return

        raw = input(f"{target_acc.nickname}으로 입금할 금액을 입력하세요: ")
        amount = self.clean_amount_input(raw)
        if amount is None:
            print("❌ 유효하지 않은 금액입니다. 숫자만 입력해주세요.")
            return

        # 기본계좌에서 출금 후 target에 입금
        if base_acc.withdraw(amount, to_account=target_acc):
            target_acc.deposit(amount, from_account=base_acc)
            self.deposit_count += 1
            # 5의 배수 시 이자 지급 (target 계좌에 지급)
            if self.deposit_count % 5 == 0:
                target_acc.add_interest()
        input("엔터를 눌러 메인으로 돌아가기 ▶ ")

    def withdraw_process(self):
        # 출금 흐름: 선택계좌 -> 기본계좌
        base_acc = self.accounts[0]
        print("\n[출금] — 선택한 계좌에서 기본 계좌로 송금합니다.")
        source_acc = self.select_account("출금")
        if not source_acc:
            return
        if source_acc == base_acc:
            print("❌ 기본 계좌에서는 출금(송금)을 할 수 없습니다.")
            return

        # 출금 계좌 비밀번호 확인
        if not self.verify_password(source_acc):
            return

        raw = input(f"{source_acc.nickname}에서 출금할 금액을 입력하세요: ")
        amount = self.clean_amount_input(raw)
        if amount is None:
            print("❌ 유효하지 않은 금액입니다. 숫자만 입력해주세요.")
            return

        if source_acc.withdraw(amount, to_account=base_acc):
            base_acc.deposit(amount, from_account=source_acc)
            time.sleep(1)
            print("💸 출금(송금)이 완료되었습니다.")
        input("엔터를 눌러 메인으로 돌아가기 ▶ ")

    def show_account_history(self):
        # 특정 계좌의 거래 내역 조회
        acc = self.select_account("조회")
        if acc:
            acc.show_history()
        input("엔터를 눌러 메인으로 돌아가기 ▶ ")


def main():
    bank = BankSystem()
    bank.show_main_menu()


if __name__ == "__main__":
    main()


===== [메인 화면] =====

[보유 계좌 목록]
1. 기본계좌 (123-45-678900) - 잔고: 100,000원

메뉴: 1. 계좌 내역  2. 입금(기존→선택)  3. 출금(선택→기본)  4. 신규 계좌 개설  5. 종료
